In [1]:
%%sh
pwd

/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/juptyer-notebooks


In [2]:
!pip install transforms3d --quiet

In [81]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import transforms3d.quaternions as quat
from scipy import signal
import scipy.fft as sfft

In [256]:
focal_length_x = 5212.5371
focal_length_y = 6255.0444
principal_point_x = 640.
principal_point_y = 512.
K = np.array([[focal_length_x, 0, principal_point_x],
                          [0, focal_length_y,principal_point_y],
                          [0, 0, 1]])
K

# range 01f0f7459b,0,299.2
#01f0f7459b,1,

laser_range_distance = 300

labels = np.array([[7.64022827,-46.98920059,-42.21747971,0.98997474,-0.0941886,-0.07856753,0.07003857],
    [12.37142944,-36.08272934,-76.19368744,0.98547542,-0.09323603,-0.13402745,0.04670797],
    [48.83045959,-62.45590591,-149.01345825,0.9463315,-0.15760925,-0.27446723,0.06544811],
    [42.9213562,3.1468811,-156.43341064,0.96294099,-0.05801675,-0.26296937,-0.0150205],
    [25.3291626,-15.59156799,-118.26628113,0.97709697,-0.05717814,-0.20439324,0.01534067],
    [55.51976013,-30.00940132,-172.91856384,0.94631386,-0.10770535,-0.30420068,0.01874543],
    [ 69.65390015,-14.23764992,-194.42814636,0.93474096,-0.0921587,-0.34309945,-0.00697307],
    [67.03590393,57.95062637,-181.86766052,0.94076264,0.03300816,-0.32461578,-0.09219801],
    [95.08578491,70.74472046,-209.08718872,0.91551983,0.02235752,-0.38302255,-0.12090118],
    [115.60142517,80.90587616,-223.3885498,0.89717531,0.01968047,-0.41800421,-0.14128494]])

In [185]:
root = '/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/'

In [186]:
# image 001:
# trans 7.64022827,-46.98920059,-42.21747971
# quat ,0.98997474,-0.0941886,-0.07856753,0.07003857
orig_trans = np.array([7.64022827,-46.98920059,-42.21747971])
orig_quat = np.array([0.98997474,-0.0941886,-0.07856753,0.07003857])

In [187]:
img1path = os.path.join(root, 'data/images/01f0f7459b/000.png')
img2path = os.path.join(root, 'data/images/01f0f7459b/001.png')

img1 = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)
img1full = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
img2full = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)

In [188]:
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img2, None)
kp2, des2 = orb.detectAndCompute(img1, None)

In [189]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

matched_keypoints1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
matched_keypoints2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

In [190]:
essential_matrix, _ = cv2.findEssentialMat(matched_keypoints1, matched_keypoints2, K)

In [191]:
_, R, t, _ = cv2.recoverPose(essential_matrix, matched_keypoints1, matched_keypoints2, K)

In [193]:
# Convert rotation vector to rotation matrix
R_mat, _ = cv2.Rodrigues(R)

# Compute translation
translation = -np.dot(R_mat.T, t)

In [220]:
# mean_squared_error(-R_mat.ravel() * t.T, orig_trans)
res = -R_mat.ravel() * t.T
res[0]
mean_absolute_error(res[0], orig_trans)

30.882302856666666

In [192]:
# x,y,z (i.e., forward-backward, left-right, up-down)
t.ravel()

array([ 0.1210821 , -0.20337419,  0.97158534])

In [77]:
scale_factor = laser_range_distance/t[2]
translation_scaled = t*scale_factor

In [94]:
print(orig_trans)
print(t.ravel())
print(translation_scaled.ravel())

[  7.64022827 -46.98920059 -42.21747971]
[ 0.1210821  -0.20337419  0.97158534]
[ 37.3869667 -62.7966004 300.       ]


In [85]:
print(mean_squared_error(np.array([0.,0.,0.]), orig_trans))
print(mean_squared_error(t.ravel(), orig_trans))
print(mean_squared_error(translation_scaled.ravel(), orig_trans))

1349.5578843897417
1370.248816996463
39415.848585019805


In [32]:
a = Rotation.from_matrix(R).as_quat()
a
# print('Quaternion test:')
print(f'{mean_absolute_error(a, orig_quat)} : recover pose')
# print(mean_absolute_error(b, orig_quat))

0.5270817984556209 : recover pose


In [309]:
start = 1
N = 10
n_beat_base = 1
for i in range(start, N):
    # 06d74ed510/
    # 01f0f7459b
    chaindir = "06d74ed510"
    img1path = os.path.join(root, f'data/images/{chaindir}/000.png')
    img2path = os.path.join(root, f'data/images/01f0f7459b/00{i}.png')
    
    img1 = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)
    
    try:
        orb = cv2.ORB_create()
        keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
        keypoints2, descriptors2 = orb.detectAndCompute(img2, None)
    
        # Match keypoints between the two images
        matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
        matches = matcher.match(descriptors1, descriptors2, None)
        
        # Extract matched keypoints
        points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        # Compute Fundamental matrix (F) and Essential matrix (E)
        F, mask = cv2.findFundamentalMat(points1, points2, cv2.RANSAC)
        E, mask = cv2.findEssentialMat(points1, points2, K)
        
        # Decompose Essential matrix to get rotation and translation
        _, R, t, _ = cv2.recoverPose(E, points1, points2, K)
        
        # zero out the z and scale by 2k
        ttmp = np.append(t[:2], 0) * 2000
        
        original_trans = labels[i-1][:3]
        original_quat = labels[i-1][3:]

        print(original_trans)
        print(ttmp)
        print(t.ravel() * 1000)
    
        base_result = mean_absolute_error(np.array([0,0,0]),original_trans)
        test_result = mean_absolute_error(ttmp,original_trans)
        if test_result < base_result:
            n_beat_base += 1
        print(f'{base_result} = BASE CASE')
        print(test_result)
        print()
    except Exception as e:
        print(e)

print(f'Frac of best : {n_beat_base / N}')

[  7.64022827 -46.98920059 -42.21747971]
[ 222.12278313 -208.33042203    0.        ]
[ 111.06139156 -104.16521102  988.33950448]
32.28230285666667 = BASE CASE
139.34708533674873

[ 12.37142944 -36.08272934 -76.19368744]
[ 5.36144895 -1.47600951  0.        ]
[ 2.68072447e+00 -7.38004753e-01 -9.99996135e+02]
41.54928207333334 = BASE CASE
39.27012925529106

[  48.83045959  -62.45590591 -149.01345825]
[-1.87499077  9.45718234  0.        ]
[-9.37495386e-01  4.72859117e+00 -9.99988381e+02]
86.76660791666666 = BASE CASE
90.5439989544373

[  42.9213562     3.1468811  -156.43341064]
[-35.21345148  38.12748642   0.        ]
[-17.60672574  19.06374321 999.66323175]
67.50054931333334 = BASE CASE
89.84960787917969

[  25.3291626   -15.59156799 -118.26628113]
[ 13.19774527 -23.6587227    0.        ]
[  6.59887264 -11.82936135 999.90825634]
53.06233724 = BASE CASE
46.15495105498322

[  55.51976013  -30.00940132 -172.91856384]
[11.90943051 -9.00367343  0.        ]
[  5.95471526  -4.50183671 999.972137

In [288]:
p=np.array([2,3,4])


In [290]:
p[2]=2

In [291]:
p

array([2, 3, 2])

In [10]:
img1path = os.path.join(root, 'data/images/01f0f7459b/000.png')
img2path = os.path.join(root, f'data/images/01f0f7459b/001.png')

img1 = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)
img1full = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
img2full = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)

orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
# Match keypoints
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
# Extract matched keypoints
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Match descriptors
matches = bf.match(des1, des2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x: x.distance)

# Extract matched keypoints
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Calculate Fundamental matrix
F, mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC)

# Select only inlier points
src_pts = src_pts[mask.ravel() == 1]
dst_pts = dst_pts[mask.ravel() == 1]

# Triangulate points using the camera intrinsic matrix
pts4D = cv2.triangulatePoints(np.eye(3), camera_matrix, src_pts.T, dst_pts.T)

# Convert homogeneous coordinates to Cartesian
pts3D = cv2.convertPointsFromHomogeneous(pts4D.T.reshape(-1, 4))

# Calculate translation vector in real-world coordinates
translation_vector = np.mean(pts3D, axis=0)

# Output translation vector
print("Translation (x, y, z) in real-world units:", translation_vector.ravel())

error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/calib3d/src/triangulate.cpp:82: error: (-209:Sizes of input arguments do not match) Size of projection matrices must be 3x4 in function 'icvTriangulatePoints'


In [135]:
def execute_test(img1, img2, orig_trans, orig_quat):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    # Match keypoints
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    # Extract matched keypoints
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Estimate homography
    # H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)

    num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, camera_matrix)
    a = Rotation.from_matrix(Rs[0]).as_quat()
    b = Rotation.from_matrix(H).as_quat()

    # print('Quaternion test:')
    # print(f'{mean_absolute_error(a, orig_quat)} : Decomposition')
    # print(mean_absolute_error(b, orig_quat))
    # print()
    print('Translation test:')
    print(f'{mean_absolute_error(np.array([0.,0.,0.]), orig_trans)} Base case')
    print(mean_absolute_error(Ts[0], orig_trans))
    print()

In [136]:
for i in range(1, 11):
    print(f'Image = {i}')
    img1path = os.path.join(root, 'data/images/01f0f7459b/000.png')
    img2path = os.path.join(root, f'data/images/01f0f7459b/00{i}.png')
    
    img1 = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)
    img1full = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
    img2full = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)

    original_trans = labels[i-1][:3]
    original_quat = labels[i-1][3:]
    
    execute_test(img1, img2, original_trans, original_quat)

Image = 1
Translation test:
32.28230285666667 Base case
33.59082343862354

Image = 2
Translation test:
41.54928207333334 Base case
45.747369978317465

Image = 3
Translation test:
86.76660791666666 Base case
91.3611142404352

Image = 4
Translation test:
67.50054931333334 Base case
53.65835905079221

Image = 5
Translation test:
53.06233724 Base case
57.6353925224469

Image = 6
Translation test:
86.14924176333334 Base case
39.95098152651341

Image = 7
Translation test:
92.77323214333335 Base case
39.434431765392404

Image = 8
Translation test:
102.28473027333332 Base case
57.15854380839394

Image = 9
Translation test:
124.97256469666667 Base case
89.59045605843141

Image = 10


[ WARN:0@3188.895] global loadsave.cpp:248 findDecoder imread_('/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/data/images/01f0f7459b/0010.png'): can't open/read file: check file path/integrity
[ WARN:0@3188.922] global loadsave.cpp:248 findDecoder imread_('/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/data/images/01f0f7459b/0010.png'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/core/src/batch_distance.cpp:274: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'batchDistance'


In [177]:
def extract_image_points(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Initialize the ORB detector
    orb = cv2.ORB_create()

    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    # Convert keypoints to numpy array
    img_points = np.float32([kp.pt for kp in keypoints])

    return img_points

In [182]:
img1path = os.path.join(root, 'data/images/01f0f7459b/000.png')
img2path = os.path.join(root, 'data/images/01f0f7459b/001.png')
img1 = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)

img_points1 = extract_image_points(img1path)
img_points2 = extract_image_points(img2path)

img_points1_norm = cv2.undistortPoints(img_points1.reshape(-1,1,2), K, None)
img_points2_norm = cv2.undistortPoints(img_points2.reshape(-1,1,2), K, None)

# Compute essential matrix
E, _ = cv2.findEssentialMat(np.float32(img_points1_norm), np.float32(img_points2_norm), K)

# E, _ = cv2.findEssentialMat(img_points1_norm, img_points2_norm, K)

# Decompose essential matrix to get rotation and translation
_, R, t, _ = cv2.recoverPose(E, img_points1_norm, img_points2_norm, K)

# Convert rotation vector to rotation matrix
R_mat, _ = cv2.Rodrigues(R)

# Compute translation
translation = -np.dot(R_mat.T, t)

error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/calib3d/src/five-point.cpp:446: error: (-215:Assertion failed) npoints >= 0 && points2.checkVector(2) == npoints && points1.type() == points2.type() in function 'findEssentialMat'


In [168]:
translation * 1000

array([ 4.50534107, -0.52868734,  0.        ])

In [152]:
Ts[0].ravel()

array([-0.75640526, -0.0809536 ,  3.25011009])

In [148]:
orig_trans

array([  7.64022827, -46.98920059, -42.21747971])

In [73]:
sift = cv.SIFT_create()

In [74]:
keypoints1, descriptor1 = sift.detectAndCompute(img1, None)
keypoints2, descriptor2 = sift.detectAndCompute(img2, None)

In [75]:
FLANN_INDEX_KDTREE = 1
indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
searchParams = dict(checks=50)
flann = cv.FlannBasedMatcher(indexParams, searchParams)
nNeighbors = 2

In [76]:
matches = flann.knnMatch(descriptor1, descriptor2, k=nNeighbors)

In [77]:
goodMatches = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        goodMatches.append(m)

minGoodMatches = 5

In [78]:
len(goodMatches)

2

In [79]:
if len(goodMatches) > minGoodMatches:
    source_points = np.float32([keypoints1[m.queryIdx].pt for m in goodMatches]).reshape(-1,1,2)
    destination_points = np.float32([keypoints2[m.queryIdx].pt for m in goodMatches]).reshape(-1,1,2)
    errorThreshold = 5
    M, mask = cv.findHomography(source_points, destination_points, cv.RANSAC, errorThreshold)
    matches_mask = mask.ravel().tolist()
    h,w = img1.shape
    img_border = np.float32([[0,0], [0, h-1], [w-1, h-1], [w-1,0]]).reshape(-1,1,2)
    warped_img_border = cv.perspectiveTransform(img_border, M)
    img2_poly = cv.polylines(img2, [np.int32(warped_img_border)], True, 255, 3, cv.LINE_AA)
else:
    print('Not enough good matches')

Not enough good matches


In [80]:
num, Rs, Ts, Ns = cv2.decomposeHomographyMat(M, camera_matrix)

In [81]:
a = Rotation.from_matrix(Rs[0]).as_quat()
b = Rotation.from_matrix(H).as_quat()

In [82]:
print(mean_absolute_error(a, orig_quat))
print(mean_absolute_error(b, orig_quat))

0.5360092775186155
0.30566994567587386


In [83]:
print(mean_absolute_error(np.array([0.,0.,0.]), orig_trans))
print(mean_absolute_error(Ts[0], orig_trans))

32.28230285666667
1088.50286222482
